In [0]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
import numpy as np

## 데이터 로드
## num_words를 통해 빈도수가 Top 1000만을 가져온다

In [0]:
from keras.datasets import reuters
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=1000, test_split=0.2)

## 무슨 단어를 무슨 숫자로 변환했는지 알려줌

In [0]:
reuters.get_word_index()

## 기본 키값이 아닌 value를 기준으로 정렬

In [0]:
import operator
sorted(reuters.get_word_index().items(), key=operator.itemgetter(1))

## X_test의 칸의 수를 100개로 고정
## train의 개수도 100으로 고정
## 100보다 많이 들어오면 100개로 자르고,
## 100보다 작으면 앞에 0을 추가해서 칸의 수를 100으로 고정


In [0]:
from keras.preprocessing import sequence
X_train_sequence = sequence.pad_sequences(X_train, maxlen=100)
X_train_sequence.shape

In [0]:
from keras.utils import to_categorical
X_train_onehot = to_categorical(X_train_sequence)
X_train_onehot.shape

In [0]:
X_test_sequence = sequence.pad_sequences(X_test, maxlen=100)
X_test_sequence.shape

In [0]:
X_test_onehot = to_categorical(X_test_sequence)
y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)

In [0]:
model = Sequential()
model.add(LSTM(units=100, input_shape=(100, 1000)))
model.add(Dropout(0.2))
model.add(Dense(46, activation='softmax'))

## lstm param 사이즈 계산 방법
- 0번째 : input size, cell size * 4 = weight
- 1번째 : cell size, cell size * 4  = memorize rate
- 2번째 : cell size * 4  = bias

## 4는 LSTM의 weight 갯수 (아래 그림의 노란색의 갯수)
![](https://i.stack.imgur.com/MLGXm.png)


In [0]:
model.summary()

In [0]:
lstm = model.get_layer("lstm_1")
print(f"{lstm.get_weights()[0].shape}")
print(f"{lstm.get_weights()[1].shape}")
print(f"{lstm.get_weights()[2].shape}")

In [0]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001), metrics=['acc'])

In [0]:
model.fit(X_train_onehot, y_train_onehot, epochs=30)

In [0]:
model.evaluate(X_test_onehot, y_test_onehot)